<a href="https://colab.research.google.com/github/russellheines/ml-experiments/blob/main/Chapter6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

6.1 Word-level one-hot encoding (toy example)

In [3]:
import numpy as np

samples = ['The cat sat on the mat.', 'The dog ate my homework']

token_index = {}
for sample in samples:
  for word in sample.split():
    if word not in token_index:
      token_index[word] = len(token_index) + 1

max_length = 10

results = np.zeros(shape=(len(samples),
                          max_length,
                          max(token_index.values()) + 1))
for i, sample in enumerate(samples):
  for j, word in list(enumerate(sample.split()))[:max_length]:
    index = token_index.get(word)
    results[i, j, index] = 1

6.3 Using Keras for word-level one-hot encoding

In [4]:
from keras.preprocessing.text import Tokenizer
samples = ['The cat sat on the mat.', 'The dog ate my homework']

tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(samples)

sequences = tokenizer.texts_to_sequences(samples)

one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9 unique tokens.


6.5 Instantiating an Embedding layer

In [5]:
from keras.layers import Embedding

embedding_layer = Embedding(1000, 64)

6.6 Loading the IMDB data for use with an Embedding layer

In [6]:
from keras.datasets import imdb
from keras import preprocessing

max_features = 10000
maxlen = 20

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

17464789/17464789 [==============================] - 0s 0us/step


6.7 Using an Embedding layer and classifier on the IMDB data

In [7]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Embedding(10000, 8, input_length=maxlen))

model.add(Flatten())

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 8)             80000     
                                                                 
 flatten (Flatten)           (None, 160)               0         
                                                                 
 dense (Dense)               (None, 1)                 161       
                                                                 
Total params: 80161 (313.13 KB)
Trainable params: 80161 (313.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 4s 4ms/step - loss: 0.6724 - acc: 0.6209 - val_loss: 0.6262 - val_acc: 0.6908
Epoch 2/10
625/625 [==============================] - 2s 3ms/step - loss: 0.5469 - acc: 0.7492 - val_loss: 0.5300 - val_acc: 0.7296
Epoch 3/10
625/625

In [9]:
test_loss, test_acc = model.evaluate(x_test, y_test)
test_acc

782/782 [==============================] - 2s 2ms/step - loss: 0.5179 - acc: 0.7555


0.7554799914360046